# Read tiles to input format VAE network

### Imports
Install tensorflow:
``%pip install tensorflow``

In [ ]:
#%pip install tensorflow

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

### Create sampling layer

In [2]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon


### Build encoder


In [37]:
latent_dim = 2
encoder_inputs = keras.Input(shape=(20, 20,3)) # enter cut-out shape (20,20,3)
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x) # to vector
x = layers.Dense(16, activation="relu")(x) # linked layer
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 20, 20, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 10, 10, 32)   896         input_11[0][0]                   
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 5, 5, 64)     18496       conv2d_8[0][0]                   
__________________________________________________________________________________________________
flatten_4 (Flatten)             (None, 1600)         0           conv2d_9[0][0]                   
____________________________________________________________________________________________

### Build decoder

In [39]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(5 * 5 * 64, activation="relu")(latent_inputs) # -- shape corresponding to encoder
x = layers.Reshape((5, 5, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)


decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 2)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1600)              4800      
_________________________________________________________________
reshape_7 (Reshape)          (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_transpose_21 (Conv2DT (None, 10, 10, 64)        36928     
_________________________________________________________________
conv2d_transpose_22 (Conv2DT (None, 20, 20, 32)        18464     
_________________________________________________________________
conv2d_transpose_23 (Conv2DT (None, 20, 20, 3)         867       
Total params: 61,059
Trainable params: 61,059
Non-trainable params: 0
_______________________________________________________

## Define VAE as model
With custom train_step

In [40]:

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


## Load Data

### mnist dataset

In [17]:
# (x_train, _), (x_test, _) = keras.datasets.mnist.load_data() # reads data as numpy.ndarray

# mnist_digits = np.concatenate([x_train, x_test], axis=0) # [70000,28,28]
# print(type(mnist_digits), np.shape(mnist_digits))

# mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255  # adds 4th dimension (bands) [70000,28,28,1]
# print(type(mnist_digits),np.shape(mnist_digits))



<class 'numpy.ndarray'> (70000, 28, 28)
<class 'numpy.ndarray'> (70000, 28, 28, 1)


### Stored cut-outs
The VAE reads the example mnist data as ndarray. Therefore: read our cut-outs to np.ndarray

#### List cut-out files on disk

In [6]:
import os

imPath = '/Users/maaikeizeboud/Documents/Data/tiles_test/'
imgs = sorted([file for file in os.listdir(imPath) if file.endswith('.tif') ]) # list with image names
len(imgs)

1000

#### Import using rasterio
Importing the .tif images using the keras ``load_img`` functions doesnt work well because it uses Pillow, which has trouble with .tif/.tiff.

The .tif files are read to numpy.ndarray in the NCHW format (bands,rows,cols). Conv2DCustomBackpropInputOp only supports NHWC (Batch Size, Height of the Image, Width of the Image and Number of Channels), so reshape data.

In [43]:
import rasterio
from rasterio.plot import reshape_as_raster, reshape_as_image

# get tile size information to initialise ndarray 
with rasterio.open(imPath + imgs[0]) as src:
#     imShape = [src.count, src.width, src.height] # returns ( bands , rows , cols)
    imShape = [src.width, src.height,src.count]  # returns ( rows , cols, bands)
# initialise ndarray    
images = np.expand_dims( np.empty(imShape) , 0).astype("float32"); # expand to 4th dimension to stack all images

# load cut-outs
for i in imgs:
    with rasterio.open(imPath + i) as src:
        imArray = src.read([1,2,3]) # reads all bands (expected 3 or 4) , returns np.ndarray (bands,rows,cols)
        imArray = reshape_as_image(imArray) # reshapes to (rows,cols,bands)
        imArray = np.expand_dims(imArray, 0).astype("float32")  # ( bands , rows , cols, 1)
        images = np.concatenate((images, imArray),axis=0) # stack data
        
print(images.shape)

(1001, 20, 20, 3)


## Train model

In [44]:
# data = mnist_digits #[70000,28,28,1]
data = images; #[1001,3,20,20]


vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

vae.fit(data, epochs=1, batch_size=128)

8/8 [==============================] - 1s 41ms/step - loss: nan - reconstruction_loss: nan - kl_loss: nan


## Display grid of sampled digits

In [ ]:
# import matplotlib.pyplot as plt


# def plot_latent_space(vae, n=30, figsize=15):
#     # display a n*n 2D manifold of digits
#     digit_size = 28
#     scale = 1.0
#     figure = np.zeros((digit_size * n, digit_size * n))
#     # linearly spaced coordinates corresponding to the 2D plot
#     # of digit classes in the latent space
#     grid_x = np.linspace(-scale, scale, n)
#     grid_y = np.linspace(-scale, scale, n)[::-1]

#     for i, yi in enumerate(grid_y):
#         for j, xi in enumerate(grid_x):
#             z_sample = np.array([[xi, yi]])
#             x_decoded = vae.decoder.predict(z_sample)
#             digit = x_decoded[0].reshape(digit_size, digit_size)
#             figure[
#                 i * digit_size : (i + 1) * digit_size,
#                 j * digit_size : (j + 1) * digit_size,
#             ] = digit

#     plt.figure(figsize=(figsize, figsize))
#     start_range = digit_size // 2
#     end_range = n * digit_size + start_range
#     pixel_range = np.arange(start_range, end_range, digit_size)
#     sample_range_x = np.round(grid_x, 1)
#     sample_range_y = np.round(grid_y, 1)
#     plt.xticks(pixel_range, sample_range_x)
#     plt.yticks(pixel_range, sample_range_y)
#     plt.xlabel("z[0]")
#     plt.ylabel("z[1]")
#     plt.imshow(figure, cmap="Greys_r")
#     plt.show()


# plot_latent_space(vae)

## Display latent space clustering

In [ ]:

# def plot_label_clusters(vae, data, labels):
#     # display a 2D plot of the digit classes in the latent space
#     z_mean, _, _ = vae.encoder.predict(data)
#     plt.figure(figsize=(12, 10))
#     plt.scatter(z_mean[:, 0], z_mean[:, 1], c=labels)
#     plt.colorbar()
#     plt.xlabel("z[0]")
#     plt.ylabel("z[1]")
#     plt.show()


# (x_train, y_train), _ = keras.datasets.mnist.load_data()
# x_train = np.expand_dims(x_train, -1).astype("float32") / 255

# plot_label_clusters(vae, x_train, y_train)